In [1]:
import cv2
import matplotlib.pyplot as plt

# 1. Inverted Image

In [3]:
# Load the image in grayscale
img = cv2.imread('/Users/lengocbinh/Desktop/Koolsoft/OCR_Main/image/1.jpg', cv2.IMREAD_GRAYSCALE)

# Invert the image
inverted_img = cv2.bitwise_not(img)

# Check if the image was loaded successfully
if img is None:
    print("Error: Image could not be loaded.")
else:
    # Invert the image
    inverted_img = cv2.bitwise_not(img)

    # Save the inverted image to a new file
    output_path = 'inverted_image.png'
    cv2.imwrite(output_path, inverted_img)

    print(f"Inverted image saved as {output_path}")

Inverted image saved as inverted_image.png


# 2. Rescalling

In [4]:
def rescale_image(image_path, output_path, scale_percent):
    # Read the image from file
    img = cv2.imread(image_path)
    
    if img is None:
        print("Error: Image could not be loaded.")
        return
    
    # Calculate new dimensions
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    new_dim = (width, height)
    
    # Resize image
    resized_img = cv2.resize(img, new_dim, interpolation=cv2.INTER_CUBIC)
    
    # Save the rescaled image
    cv2.imwrite(output_path, resized_img)
    print(f"Rescaled image saved as {output_path}")

# Example usage
image_path = '/Users/lengocbinh/Desktop/Koolsoft/OCR_Main/image/1.jpg'  # Replace with your image path
output_path = 'rescaled_image.png'  # Replace with your desired output path
current_dpi = 150  # Replace with the current DPI of your image
desired_dpi = 300  # Tesseract works best with 300 DPI

# Calculate the scale percent
scale_percent = (desired_dpi / current_dpi) * 100

rescale_image(image_path, output_path, scale_percent)

Rescaled image saved as rescaled_image.png


# 3. Binarization

In [37]:
# Load the image in grayscale
img = cv2.imread('/Users/lengocbinh/Desktop/Koolsoft/OCR_Main/image/2.jpg', cv2.IMREAD_GRAYSCALE)

# Apply adaptive Gaussian thresholding
adaptive_thresh_gaussian = cv2.adaptiveThreshold(
    img, 
    255, 
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
    cv2.THRESH_BINARY, 
    11, 
    15
)

# Save the results to files
cv2.imwrite('adaptive_gaussian_threshold_2.jpg', adaptive_thresh_gaussian)


True

In [22]:
import cv2
import pytesseract
from PIL import Image

# Load the image
image_path = "/Users/lengocbinh/Desktop/Koolsoft/OCR_Main/image/1.jpg"
image = cv2.imread(image_path)

# Convert the image to RGB (OpenCV uses BGR by default)
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Perform OCR on the image
h, w, _ = rgb_image.shape
data = pytesseract.image_to_data(rgb_image, output_type=pytesseract.Output.DICT)

# Initialize variables to hold line information
current_line = -1
line_boxes = []

# Loop over each detected text item and group them by line
for i in range(len(data['level'])):
    line_number = data['line_num'][i]

    if line_number != current_line:
        # If we encounter a new line, draw the bounding box for the previous line
        if line_boxes:
            x_min = min([box[0] for box in line_boxes])
            y_min = min([box[1] for box in line_boxes])
            x_max = max([box[2] for box in line_boxes])
            y_max = max([box[3] for box in line_boxes])
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            line_boxes = []

        current_line = line_number

    # Append the current box to the list for the current line
    (x, y, width, height) = (data['left'][i], data['top'][i], data['width'][i], data['height'][i])
    line_boxes.append((x, y, x + width, y + height))

# Draw the bounding box for the last line
if line_boxes:
    x_min = min([box[0] for box in line_boxes])
    y_min = min([box[1] for box in line_boxes])
    x_max = max([box[2] for box in line_boxes])
    y_max = max([box[3] for box in line_boxes])
    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

# Save the image with bounding boxes
output_image_path = "line_bounding_boxes.png"
cv2.imwrite(output_image_path, image)

# Display the image
Image.open(output_image_path).show()


In [23]:
import cv2
import pytesseract
from PIL import Image
import numpy as np

# Load the image
image_path = "/Users/lengocbinh/Desktop/Koolsoft/OCR_Main/image/1.jpg"
image = cv2.imread(image_path)

# Convert the image to RGB (OpenCV uses BGR by default)
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Use Tesseract to find the orientation of the text
osd = pytesseract.image_to_osd(rgb_image)
angle = int(osd.split("\n")[2].split(": ")[1])
script = osd.split("\n")[3].split(": ")[1]

# Print the detected orientation and script
print(f"Detected orientation: {angle} degrees")
print(f"Detected script: {script}")

# Deskew the image
if angle != 0:
    # Compute the rotation matrix
    (h, w) = rgb_image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, -angle, 1.0)

    # Rotate the image to deskew it
    deskewed_image = cv2.warpAffine(rgb_image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    # Convert back to BGR for saving with OpenCV
    deskewed_image_bgr = cv2.cvtColor(deskewed_image, cv2.COLOR_RGB2BGR)

    # Save the deskewed image
    output_image_path = "deskewed_image.png"
    cv2.imwrite(output_image_path, deskewed_image_bgr)

    # Display the deskewed image
    Image.open(output_image_path).show()

else:
    print("No rotation needed. The image is already properly oriented.")


Detected orientation: 0 degrees
Detected script: 16.43
No rotation needed. The image is already properly oriented.
